In [1]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text

# Example usage
pdf_path = 'PSR.pdf'  # Provide the path to your PDF file
pdf_text = extract_text_from_pdf(pdf_path)
print(pdf_text[:1000])  # Print the first 1000 characters to check


LAGOS ST A TE GOVERNMENT P UBLIC  SERVICE  RULES   
REVISED TO 1st JANUAR Y , 
Lagos State Gove rnment Public Service Rules For eword FOREWORD  
It seems trite to contend that there is nothing that somewhat  mirrors the image, 
perception and indeed the effectiveness  of any Government li ke the image, 
perception and ef fectiveness of its Public Service. I have heard people refer to the 
word "bureaucracy" as if it was itself a bad thing. Checking up on the meaning of 
the word confirms what I already knew . It refers to a body of rules, procedures and 
processes by which things are done. In my opinion , this is a good thing. A  
lar ge workforce like the Lagos State Public Service, inclusive of its Local 
Government employees, numerous Parastatals and Commissions comprising over 
one hundred thousand people, employed to relate with members of the public, 
provide services and solve problems, require Rules of Engagement.  
Without  clear processes, rules and procedures, their conduct 

In [2]:
from langchain.text_splitter import CharacterTextSplitter

def split_text_into_chunks(text, chunk_size=1000, overlap=0):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    chunks = text_splitter.split_text(text)
    return chunks

# Example usage
chunks = split_text_into_chunks(pdf_text)
print(f"First 2 chunks: {chunks[:2]}")


First 2 chunks: ['LAGOS ST A TE GOVERNMENT P UBLIC  SERVICE  RULES   \nREVISED TO 1st JANUAR Y , \nLagos State Gove rnment Public Service Rules For eword FOREWORD  \nIt seems trite to contend that there is nothing that somewhat  mirrors the image, \nperception and indeed the effectiveness  of any Government li ke the image, \nperception and ef fectiveness of its Public Service. I have heard people refer to the \nword "bureaucracy" as if it was itself a bad thing. Checking up on the meaning of \nthe word confirms what I already knew . It refers to a body of rules, procedures and \nprocesses by which things are done. In my opinion , this is a good thing. A  \nlar ge workforce like the Lagos State Public Service, inclusive of its Local \nGovernment employees, numerous Parastatals and Commissions comprising over \none hundred thousand people, employed to relate with members of the public, \nprovide services and solve problems, require Rules of Engagement.  \nWithout  clear processes, rules

In [3]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings  # You can use HuggingFace embeddings instead

# 3. Create FAISS index from text chunks
def create_faiss_index(chunks):
    # Using HuggingFace Embeddings as an example (you can choose other embeddings if needed)
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    
    faiss_index = FAISS.from_texts(chunks, embeddings)
    faiss_index.save_local("faiss_index")
    return faiss_index

# Example usage
faiss_index = create_faiss_index(chunks)


C:\Users\AJACENT\AppData\Local\Temp\ipykernel_16844\2935726828.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\AJACENT FILES\GENERATIVE AI\Implementations\PSR_CHATBOT\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# 4. Query FAISS index for the most relevant documents
def query_faiss_index(query, faiss_index):
    # Retrieve the top 3 most relevant documents based on the query
    docs = faiss_index.similarity_search(query, k=3)
    
    # Combine the content of the retrieved documents to use as context for Ollama
    context = " ".join([doc.page_content for doc in docs])
    
    return context

# Example usage
query = "What is the defendant's argument in this case?"
context = query_faiss_index(query, faiss_index)
print(f"Context retrieved from FAISS: {context}")


Context retrieved from FAISS: LAGOS ST A TE GOVERNMENT P UBLIC  SERVICE  RULES   
REVISED TO 1st JANUAR Y , 
Lagos State Gove rnment Public Service Rules For eword FOREWORD  
It seems trite to contend that there is nothing that somewhat  mirrors the image, 
perception and indeed the effectiveness  of any Government li ke the image, 
perception and ef fectiveness of its Public Service. I have heard people refer to the 
word "bureaucracy" as if it was itself a bad thing. Checking up on the meaning of 
the word confirms what I already knew . It refers to a body of rules, procedures and 
processes by which things are done. In my opinion , this is a good thing. A  
lar ge workforce like the Lagos State Public Service, inclusive of its Local 
Government employees, numerous Parastatals and Commissions comprising over 
one hundred thousand people, employed to relate with members of the public, 
provide services and solve problems, require Rules of Engagement.  
Without  clear processes, rules 

In [5]:
import ollama

# Optionally, you can set your API key if Ollama requires it
# ollama.set_api_key("your_api_key")  # Uncomment and add your API key if needed

# Function to query the Ollama model
def ask_ollama(query):
    # Send the query to the downloaded model (deepseek-r1:8b)
    response = ollama.chat(model="deepseek-r1:8b", messages=[{"role": "user", "content": query}])
    return response['text']



In [6]:
def ask_question_with_ollama(query):
    # Retrieve relevant document chunks from the FAISS vector store
    docs = faiss_index.similarity_search(query, k=3)  # Retrieve top 3 relevant chunks
    context = " ".join([doc.page_content for doc in docs])  # Combine relevant chunks into a single string
    
    # Prepare the full query for Ollama
    full_query = f"Context: {context} Question: {query}"
    
    # Get the answer from Ollama
    response = ask_ollama(full_query)
    return response

# Example usage
query = "What is the defendant's argument in this case?"
response = ask_question_with_ollama(query)
print(f"Ollama's Answer: {response}")


ResponseError: model requires more system memory (5.5 GiB) than is available (2.8 GiB) (status code: 500)

In [ ]:
import ollama
available_models = ollama.list()
print(available_models)


models=[Model(model='deepseek-r1:8b', modified_at=datetime.datetime(2025, 1, 29, 12, 27, 48, 40095, tzinfo=TzInfo(+01:00)), digest='28f8fd6cdc677661426adab9338ce3c013d7e69a5bea9e704b364171a5d61a10', size=4920738407, details=ModelDetails(parent_model='', format='gguf', family='llama', families=['llama'], parameter_size='8.0B', quantization_level='Q4_K_M')), Model(model='gemma2:2b', modified_at=datetime.datetime(2025, 1, 22, 12, 53, 40, 93400, tzinfo=TzInfo(+01:00)), digest='8ccf136fdd5298f3ffe2d69862750ea7fb56555fa4d5b18c04e3fa4d82ee09d7', size=1629518495, details=ModelDetails(parent_model='', format='gguf', family='gemma2', families=['gemma2'], parameter_size='2.6B', quantization_level='Q4_0'))]
